In [1]:
%load_ext lab_black

In [2]:
# Import all the necessary packages
import pandas as pd
import numpy as np

import statsmodels.api as sm
import scipy.stats as st

import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns

import json
import requests
import yaml
from twython import Twython
from twython import Twython, TwythonError

from pandas import json_normalize

from datetime import datetime
from dateutil import parser

In [3]:
dorsey = pd.read_csv("dorsey.csv")
twitter = pd.read_csv("TWTR.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'dorsey.csv'

In [ ]:
dorsey.head(5)

In [ ]:
columns = ["created_at", "id", "full_text", "retweet_count", "favorite_count"]
dorsey = dorsey[columns]
dorsey["created_at"] = pd.to_datetime(dorsey["created_at"]).dt.date

In [ ]:
dorsey.head()

In [ ]:
dorsey.dtypes

In [ ]:
import re

# Clean The Data
def cleantext(text):
    text = re.sub(r"@[A-Za-z0-9]+", "", text)  # Remove Mentions
    text = re.sub(r"#", "", text)  # Remove Hashtags Symbol
    text = re.sub(r"RT[\s]+", "", text)  # Remove Retweets
    text = re.sub(r"https?:\/\/\S+", "", text)  # Remove The Hyper Link

    return text


# Clean The Text
dorsey["full_text"] = dorsey["full_text"].apply(cleantext)
dorsey.head()

In [4]:
from textblob import TextBlob

# Get The Subjectivity
def sentiment_analysis(ds):
    sentiment = TextBlob(ds).sentiment
    return pd.Series([sentiment.subjectivity, sentiment.polarity])


# Adding Subjectivity & Polarity
dorsey[["subjectivity", "polarity"]] = dorsey["full_text"].apply(sentiment_analysis)
dorsey.head()

NameError: name 'dorsey' is not defined

In [ ]:
df_a = (
    dorsey.groupby(["created_at"])
    .agg(
        agg_string=("full_text", "sum"),
        agg_retweet=("retweet_count", "sum"),
        agg_count=("favorite_count", "sum"),
        subjectivity_mean=("subjectivity", "mean"),
        polarity_mean=("polarity", "mean"),
    )
    .reset_index()
)
df_a.head()

In [ ]:
from wordcloud import WordCloud

allwords = " ".join([twts for twts in dorsey["full_text"]])
wordCloud = WordCloud(
    width=1000, height=1000, random_state=21, max_font_size=119
).generate(allwords)
plt.figure(figsize=(20, 20), dpi=80)
plt.imshow(wordCloud, interpolation="bilinear")
plt.axis("off")
plt.show()

In [7]:
sentiment = TextBlob("not sure anyone has heard but, I resigned from Twitter").sentiment
print(sentiment)

Sentiment(polarity=-0.25, subjectivity=0.8888888888888888)


In [ ]:
df_a.columns = [
    "Date",
    "agg_text",
    "agg_retweet",
    "agg_count",
    "subjectivity",
    "polarity",
]

In [ ]:
d = df_a.iloc[:]["Date"]
d

In [ ]:
df = pd.merge(twitter, df_a, on="Date", how="outer")

In [ ]:
df

In [ ]:
df_a

In [ ]:
twitter["Date"] = pd.to_datetime(twitter["Date"]).dt.date

In [ ]:
twitter.dtypes